# Create field point files

Description: Creates geojson files for field points.

Author: Ivan Zvonkov

Date: Nov 13, 2023


In [4]:
!pip install geopandas -q

You should consider upgrading via the '/Users/izvonkov/nasaharvest/street2sat/venv/bin/python -m pip install --upgrade pip' command.


In [76]:
import pandas as pd
import geopandas as gpd

from pathlib import Path

In [54]:
df = pd.read_csv("../data/street2sat-v2-points-in-field.csv")

In [55]:
predictions = pd.json_normalize(df["results"].apply(eval))

In [56]:
crops = list(predictions.columns[1:])

In [57]:
df = pd.concat([df, predictions], axis=1)

In [58]:
df["dominant_crop"] = df[crops].apply(lambda x: max(dict(x), key=dict(x).get), axis=1)

In [72]:
# Rename some columns
df["key"] = df["name"]
df["name"] = df["dominant_crop"]


In [68]:
df["road_lat"] = df["coord"].map(lambda coord: eval(coord)[0])
df["road_lon"] = df["coord"].map(lambda coord: eval(coord)[1])

In [73]:
def generate_gdf(cols, lon, lat):
    points = gpd.points_from_xy(df[lon], df[lat], crs="EPSG:4326")
    gdf = gpd.GeoDataFrame(df[cols], geometry=points)
    return gdf

road_cols = ["input_img", "name", "dominant_crop", "segmentation_images"] + crops
bing_cols = ["input_img", "name", "dominant_crop", "bing_img_source", "bing_img_segmentation"] + crops
mapbox_cols = ["input_img", "name", "dominant_crop", "mapbox_img_source", "mapbox_img_segmentation"] + crops

gdf_road = generate_gdf(cols=road_cols, lat="road_lat", lon="road_lon")
gdf_bing = generate_gdf(cols=bing_cols, lat="bing_field_coord_lat", lon="bing_field_coord_lon")
gdf_mapbox = generate_gdf(cols=mapbox_cols, lat="mapbox_field_coord_lat", lon="mapbox_field_coord_lon")

In [77]:
Path("../data/points").mkdir(exist_ok=True)

In [78]:
gdf_road.to_file("../data/points/road.geojson", driver="GeoJSON")
gdf_bing.to_file("../data/points/field_bing.geojson", driver="GeoJSON")
gdf_mapbox.to_file("../data/points/field_mapbox.geojson", driver="GeoJSON")